In [ ]:
# module import
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import matplotlib.tri as tri
import cartopy.crs as ccrs
import pyproj
import matplotlib.ticker as mticker
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import matplotlib.path as mpath
import seaborn as sns
import datetime
import cftime
from matplotlib import gridspec
import cartopy.feature as cft
import pandas as pd
import string
import cmocean as cm
import math
import proplot as pplt
import xesmf as xe
import netCDF4 as nc
from matplotlib.offsetbox import TextArea, DrawingArea, OffsetImage, AnnotationBbox
import matplotlib.image as mpimg
import pyproj

## OBSERVATIONS

### ATMOSPHERE

In [ ]:
netCDF_file = f'/Users/jcaillet/Documents/Elmer/CMIP6/INTERNAL_VARIABILITY_MULTIMODEL/ERA5_Tair_2m.nc'
OBS = xr.open_mfdataset(netCDF_file)
OBS = OBS.mean('time')
OBS = OBS.rename({'longitude':'lon','latitude':'lat'})
OBS['t2m']=OBS['t2m']-273.15

    # open BedMachine file for mask
netCDF_file1 = '/Users/jcaillet/Documents/Elmer/DATA/BATHY/BedMachineAntarctica-v3.nc'
BATHY = xr.open_mfdataset(netCDF_file1)
BATHY = BATHY.sortby('y')

    # dégradation BedMachine de 500 m à 5 km
dx = np.arange(-3333000,3333001,5000)
dy = np.arange(-3333000,3333001,5000)
da = xr.DataArray(data=np.ones((dy.size,dx.size)),
                  dims=["x", "y"],
                  coords=dict(x=(["x"], dx), y=(["y"], dy),))
BATHY = BATHY.interp_like(da) 

    # add lat,lon
cproj='epsg:3031'
p = pyproj.Proj(cproj)
xin ,yin = np.meshgrid(BATHY.x,BATHY.y)
lon,lat = p(xin,yin,inverse=True)
BATHY = BATHY.assign_coords(lon=(("y","x"), lon))
BATHY = BATHY.assign_coords(lat=(("y","x"), lat))

    # mask atmo (ice sheet (floating+grounded part))
BATHY['mask_atmo']= BATHY['mask']-BATHY['mask']
BATHY['mask_atmo']= xr.where(BATHY['mask']==0,0,10)

# interpolation of ERA5 grid on BedMachine Grid
ds = OBS['t2m']
ds_out = BATHY['mask_atmo']
regridder = xe.Regridder(ds, ds_out, "bilinear")
dr_out = regridder(ds)
BATHY['t2m']=dr_out 
    
# calculation of mean value for atmosphere (regular grid)
BATHY['mean']= BATHY['t2m'].where(BATHY['mask_atmo']==10).mean(['x','y'])
meanOBS = float(BATHY['mean'])


### OCEAN

In [ ]:
netCDF_filei = '/Users/jcaillet/Documents/Elmer/CMIP6/OBSERVATIONS/T/obs_temperature_1995-2017_8km_x_60m.nc'
OBSTC = xr.open_mfdataset(netCDF_filei)

# open BedMachine file for mask
netCDF_file1 = '/Users/jcaillet/Documents/Elmer/DATA/BATHY/BedMachineAntarctica-v3.nc'
BATHY = xr.open_mfdataset(netCDF_file1)
BATHY = BATHY.sortby('y')

# dégradation BedMachine de 500 m à 5 km
dx = np.arange(-3333000,3333001,5000)
dy = np.arange(-3333000,3333001,5000)
da = xr.DataArray(data=np.ones((dy.size,dx.size)),
                  dims=["x", "y"],
                  coords=dict(x=(["x"], dx), y=(["y"], dy),))
BATHY = BATHY.interp_like(da) 

# add lat,lon
cproj='epsg:3031'
p = pyproj.Proj(cproj)
xin ,yin = np.meshgrid(BATHY.x,BATHY.y)
lon,lat = p(xin,yin,inverse=True)
BATHY = BATHY.assign_coords(lon=(("y","x"), lon))
BATHY = BATHY.assign_coords(lat=(("y","x"), lat))

# mask ocean (continental shelf : z > -1500m, ocean part, lat < 60°S)
BATHY['mask_ocean']= BATHY['mask']-BATHY['mask']
BATHY['mask_ocean']= xr.where(BATHY['mask']==0,10,0)
#BATHY['mask_ocean']= xr.where(BATHY['mask']==3,10,BATHY['mask_ocean'].values)
BATHY['mask_ocean']= xr.where(BATHY['bed']>-1500,BATHY['mask_ocean']+10,BATHY['mask_ocean'].values)
BATHY['mask_ocean']= xr.where(BATHY['lat']<-60,BATHY['mask_ocean']+10,BATHY['mask_ocean'].values)

# remove values on the land and beyond the bed elevation (using mask of BedMachinev3)
bed_interp = BATHY.interp_like(OBSTC) # interpolation on the observed grid
OBSTC['temperature'] = OBSTC.temperature.where(bed_interp.mask==0)
OBSTC['bed'] = OBSTC['lon'] - OBSTC['lon'] + bed_interp.bed.values
OBSTC['temp_corr'] = xr.where(OBSTC['bed'] < OBSTC['z'], OBSTC['temperature'],np.nan)
OBSTC['mask_nan'] = OBSTC['temp_corr'] - OBSTC['temp_corr']
OBSTC['mask_nan'] = xr.where(OBSTC['temp_corr']>-5, 1,0)
OBSTC['mask_ocean'] = OBSTC['lon'] - OBSTC['lon'] + bed_interp.mask_ocean.values

# calculation of mean value over 200-700m
OBSTC['temp_200700'] = OBSTC['temp_corr'].where((OBSTC['mask_nan']==1) & (OBSTC['z']>-700) & (OBSTC['z']<-200)).mean('z')
OBSTC['mean'] = OBSTC['temp_200700'].where(OBSTC['mask_ocean']==30).mean(['x','y'])
meanOBSO=float(OBSTC['mean'])

# CMIP6 MODELS

In [ ]:
## CREATE A XARRAY DATASET
name = np.array(['ACCESS-ESM1-5','CESM2','CNRM-CM6-1','CNRM-ESM2-1','CanESM5','GISS-E2-1-G','GISS-E2-1-H','INM-CM5-0','IPSL-CM6A-LR','MIROC-ES2L','MIROC6','MPI-ESM1-2-HR','MPI-ESM1-2-LR','NorCPM1','UKESM1-0-LL'])
#name = np.array(['IPSL-CM6A-LR','UKESM1-0-LL'])

DATA = xr.Dataset(data_vars=dict(
                        Tmean_atmo=(["name"], np.ones(name.size)),
                        Tstd_atmo=(["name"], np.ones(name.size)),
                        Tmean_ocean=(["name"], np.ones(name.size)),
                        Tstd_ocean=(["name"], np.ones(name.size)),
                        ),
                  coords=dict(name=(["name"], name)),
                 )

## MASK 
    # open BedMachine file for mask
netCDF_file1 = '/Users/jcaillet/Documents/Elmer/DATA/BATHY/BedMachineAntarctica-v3.nc'
BATHY = xr.open_mfdataset(netCDF_file1)
BATHY = BATHY.sortby('y')

    # dégradation BedMachine de 500 m à 5 km
dx = np.arange(-3333000,3333001,5000)
dy = np.arange(-3333000,3333001,5000)
da = xr.DataArray(data=np.ones((dy.size,dx.size)),
                  dims=["x", "y"],
                  coords=dict(x=(["x"], dx), y=(["y"], dy),))
BATHY = BATHY.interp_like(da) 

    # add lat,lon
cproj='epsg:3031'
p = pyproj.Proj(cproj)
xin ,yin = np.meshgrid(BATHY.x,BATHY.y)
lon,lat = p(xin,yin,inverse=True)
BATHY = BATHY.assign_coords(lon=(("y","x"), lon))
BATHY = BATHY.assign_coords(lat=(("y","x"), lat))

    # mask ocean (continental shelf : z > -1500m, ocean part, lat < 60°S)
BATHY['mask_ocean']= BATHY['mask']-BATHY['mask']
BATHY['mask_ocean']= xr.where(BATHY['mask']==0,10,0)
#BATHY['mask_ocean']= xr.where(BATHY['mask']==3,10,BATHY['mask_ocean'].values)
BATHY['mask_ocean']= xr.where(BATHY['bed']>-1500,BATHY['mask_ocean']+10,BATHY['mask_ocean'].values)
BATHY['mask_ocean']= xr.where(BATHY['lat']<-60,BATHY['mask_ocean']+10,BATHY['mask_ocean'].values)
    # mask atmo (ice sheet (floating+grounded part))
BATHY['mask_atmo']= BATHY['mask']-BATHY['mask']
BATHY['mask_atmo']= xr.where(BATHY['mask']==0,0,10)


## Calculation of mean and std
name0 = np.array(['ACCESS-ESM1-5','CESM2','CNRM-CM6-1','CNRM-ESM2-1','CanESM5','GISS-E2-1-G','GISS-E2-1-H','INM-CM5-0','IPSL-CM6A-LR','MIROC-ES2L','MIROC6','MPI-ESM1-2-HR','MPI-ESM1-2-LR','NorCPM1','UKESM1-0-LL'])
r=-1
for n in name:
    r=r+1
    name=name0[r]
    print(r)
    # open CMIP file
    netCDF_file = f'/Users/jcaillet/Documents/Elmer/CMIP6/INTERNAL_VARIABILITY_MULTIMODEL/Tatmo_{name}.nc'
    ATMO = xr.open_mfdataset(netCDF_file)
    ATMO = ATMO.rename({'x':'lon','y':'lat'})
    ATMO['mask_atmo']=ATMO['area']-ATMO['area']
    ATMO['T_atmo']=ATMO['T_atmo']-273.15
    
    netCDF_fileo = f'/Users/jcaillet/Documents/Elmer/CMIP6/INTERNAL_VARIABILITY_MULTIMODEL/Tocean_{name}.nc'
    OCEAN = xr.open_mfdataset(netCDF_fileo)
    #if ((r == 0) OR (r == 4) OR (r == 9) OR (r == 10) OR (r == 11) OR (r == 12) OR (r == 13) OR (r == 14)):
    if r == 0 or r == 4 or r == 9 or r == 10 or r == 11 or r == 12 or r == 13 or r == 14:
        OCEAN = OCEAN.rename({'longitude':'lon','latitude':'lat'})
    elif r == 5 or r == 6 or r == 7:
        OCEAN = OCEAN.rename({'x':'lon','y':'lat'})
        OCEAN = OCEAN.rename_dims({'lat':'y','lon':'x'})
    elif r == 8 :
        OCEAN = OCEAN.rename({'nav_lon':'lon','nav_lat':'lat'})
    
    OCEAN['mask_ocean']=OCEAN['area']-OCEAN['area']
    
    # interpolation of mask on CMIP6 grid
    ds = BATHY['mask_atmo']
    ds_out = ATMO['area']
    regridder = xe.Regridder(ds, ds_out, "nearest_s2d")
    dr_out = regridder(ds)
    ATMO['mask_atmo']=dr_out 
    
    ds = BATHY['mask_ocean']
    ds_out = OCEAN['area']
    regridder = xe.Regridder(ds, ds_out, "nearest_s2d")
    dr_out = regridder(ds)
    OCEAN['mask_ocean']=dr_out
    
    # calculation of mean and std for atmosphere
    ATMO['value_memb']=(ATMO['T_atmo']*ATMO['area']).where(ATMO['mask_atmo']==10).sum(['lat','lon'])/ATMO['area'].where(ATMO['mask_atmo']==10).sum(['lat','lon'])
    mean = float(ATMO['value_memb'].mean('num'))
    std = float(ATMO['value_memb'].std('num'))
    
    DATA['Tmean_atmo'][r]=mean
    DATA['Tstd_atmo'][r]=std
    
    # calculation of mean and std for atmosphere
    OCEAN['value_memb']=(OCEAN['T_ocean']*OCEAN['area']).where(OCEAN['mask_ocean']==30).sum(['y','x'])/OCEAN['area'].where(OCEAN['mask_ocean']==30).sum(['y','x'])
    mean = float(OCEAN['value_memb'].mean('num'))
    std = float(OCEAN['value_memb'].std('num'))
    
    DATA['Tmean_ocean'][r]=mean
    DATA['Tstd_ocean'][r]=std

In [ ]:
sns.set_context('paper')

markers = ["$ACCESS$", "$CESM$", "$CNRM$", "$CNRM$", "$CanESM$", "$GISS$", "$GISS$", "$INM$", "$IPSL$", "$MIROC$", "$MIROC$", "$MPI$", "$MPI$", "$NorCPM$", "$UKESM$"]
markersA = ["$ACCESS$", "", "$CNRM$", "$CNRM$", "", "", "$GISS$", "$INM$", "$IPSL$", "$MIROC$", "$MIROC$", "$MPI$", "$MPI$", "$NorCPM$", "$UKESM$"]
markersB = ["", "$CESM$", "", "$CNRM$", "$CanESM$", "$GISS$", "$GISS$", "$INM$", "$IPSL$", "$MIROC$", "$MIROC$", "$MPI$", "$MPI$", "", "$UKESM$"]
color=['gold','peru','lightgreen','limegreen','silver','lightskyblue','dodgerblue','turquoise','green','darkorange','red','magenta','blueviolet','pink','blue']
name00 = np.array(['ACCESS-ESM1-5 (40/18)','CESM2 (11)','CNRM-CM6-1 (30)','CNRM-ESM2-1 (11)','CanESM5 (25)','GISS-E2-1-G (10)','GISS-E2-1-H (10)','INM-CM5-0 (10)','IPSL-CM6A-LR (33)','MIROC-ES2L (27/10)','MIROC6 (50)','MPI-ESM1-2-HR (10)','MPI-ESM1-2-LR (10)','NorCPM1 (30)','UKESM1-0-LL (16)'])


f = plt.figure(figsize = (8, 4))
ax={}
nb_rows = 1
nb_cols = 2
   
ax1 = f.add_subplot(nb_rows,nb_cols, 1)
ax1.vlines(meanOBS, 0.05, 0.275, colors='k', linestyles='--', label='ERA5/ISMIP6 climatology')
for i in np.arange(0,DATA.name.size,1):
    c = ax1.scatter(DATA['Tmean_atmo'][i], DATA['Tstd_atmo'][i], s=30, c=color[i], label=name00[i])

for i, label in enumerate(markersA):
    ax1.annotate(label, (DATA['Tmean_atmo'][i]-0.4, DATA['Tstd_atmo'][i]+0.005), fontsize=7, c=color[i])

ax1.annotate('CESM', (DATA['Tmean_atmo'][1]-0.4, DATA['Tstd_atmo'][1]-0.01), fontsize=7, c='peru')
ax1.annotate('GISS', (DATA['Tmean_atmo'][5]-0.4, DATA['Tstd_atmo'][5]-0.01), fontsize=7, c='lightskyblue')
ax1.annotate('CanESM', (DATA['Tmean_atmo'][4]-1, DATA['Tstd_atmo'][4]+0.005), fontsize=7, c='silver')
    
ax1.set_title('Air Temperature over Antarctica', fontsize=10)
ax1.set_xlabel('Multi-member mean (°C)', fontsize=10)
ax1.set_ylabel('Multi-member standard deviation (°C)', fontsize=10)
ax1.set_xlim(-37,-23)
ax1.set_ylim(0.05,0.275)
plt.grid(True)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.legend(fontsize=6.5, loc="upper right")


ax2 = f.add_subplot(nb_rows,nb_cols, 2)
ax2.vlines(meanOBSO, 0.01, 0.13, colors='k', linestyles='--', label='ISMIP6:1995-2017')
for i in np.arange(0,DATA.name.size,1):
    c = ax2.scatter(DATA['Tmean_ocean'][i], DATA['Tstd_ocean'][i], s=30, c=color[i])
    
for i, label in enumerate(markersB):
    ax2.annotate(label, (DATA['Tmean_ocean'][i]-0.1, DATA['Tstd_ocean'][i]+0.002), fontsize=7, c=color[i])
    
ax2.annotate('ACCESS', (DATA['Tmean_ocean'][0]-0.2, DATA['Tstd_ocean'][0]-0.005), fontsize=7, c='gold')
ax2.annotate('NorCPM', (DATA['Tmean_ocean'][13], DATA['Tstd_ocean'][13]+0.002), fontsize=7, c='pink')
ax2.annotate('CNRM', (DATA['Tmean_ocean'][2]-0.1, DATA['Tstd_ocean'][2]-0.005), fontsize=7, c='lightgreen')

ax2.set_title('Ocean Temperature over the continental shelf', fontsize=10)
ax2.set_xlabel('Multi-member mean (°C)', fontsize=10)
ax2.set_ylabel('Multi-member standard deviation (°C)', fontsize=10)
ax2.set_xlim(-1.25,2)
ax2.set_ylim(0.01,0.13)
plt.grid(True)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

plt.tight_layout()
plt.savefig('/Users/jcaillet/Documents/Elmer/CMIP6/FIGURES/Figure_comparaisonCMIP6.pdf')